# Import Libraries

In [ ]:

import numpy as np 
import pandas as pd 
import os, xgboost, catboost
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder, RobustScaler
le = LabelEncoder()
pd.set_option('display.max_colwidth',2500)
pd.set_option('display.max_columns',500)

# Load Data

In [ ]:
rp = pd.read_csv('data/videos_watched_on_repeat.csv')
ss = pd.read_csv('data/SampleSubmission.csv')
df = pd.read_csv('data/train.csv/train.csv')
tdf = pd.read_csv('data/test.csv/test.csv')

In [ ]:
c8 = pd.read_csv('cluster/fr8.csv')
c2 = pd.read_csv('cluster/fr2.csv')
c1 = pd.read_csv('cluster/fr1.csv')
c6 = pd.read_csv('cluster/fr6.csv')
c9 = pd.read_csv('cluster/fr9.csv')
c3 = pd.read_csv('cluster/fr3.csv')
c5 = pd.read_csv('cluster/fr5.csv')
c7 = pd.read_csv('cluster/fr7.csv')
c4 = pd.read_csv('cluster/fr4.csv')

In [ ]:
clust = c1.append(c2.append(c3.append(c4.append(c5.append(c6.append(c7.append(c8.append(c9))))))))
clust = clust.drop_duplicates(['vid_id','session_id'])
clust['played'] = clust.groupby('session_id')['session_id'].transform('count')
clust.describe()

In [ ]:
clust = clust[clust.played>4]
clust.describe()

In [ ]:
clust = pd.merge(clust,df[['vid_id','target']],on='vid_id',how='left')
clust.describe()

In [ ]:
clust['target'] = clust.groupby('session_id')['target'].transform('mean')
clust.describe()

In [ ]:
clust = clust.pivot(index='vid_id', values='target', columns=['session_id'])
clust

In [ ]:
clust = clust.reset_index()
clust['estimated_target'] = clust.mean(axis=1)
clust.describe()

In [ ]:
clust = clust.reset_index()
clust['estd'] = clust.std(axis=1)
clust = clust[['vid_id','estimated_target','estd']]
clust.describe()

In [ ]:
fr8 = pd.read_csv('preprocess/fr8.csv')
fr2 = pd.read_csv('preprocess/fr2.csv')
fr1 = pd.read_csv('preprocess/fr1.csv')
fr6 = pd.read_csv('preprocess/fr6.csv')
fr9 = pd.read_csv('preprocess/fr9.csv')
fr3 = pd.read_csv('preprocess/fr3.csv')
fr5 = pd.read_csv('preprocess/fr5.csv')
fr7 = pd.read_csv('preprocess/fr7.csv')
fr4 = pd.read_csv('preprocess/fr4.csv')

In [ ]:
fr8m = pd.read_csv('sessmean/fr8.csv')
fr2m = pd.read_csv('sessmean/fr2.csv')
fr1m = pd.read_csv('sessmean/fr1.csv')
fr6m = pd.read_csv('sessmean/fr6.csv')
fr9m = pd.read_csv('sessmean/fr9.csv')
fr3m = pd.read_csv('sessmean/fr3.csv')
fr5m = pd.read_csv('sessmean/fr5.csv')
fr7m = pd.read_csv('sessmean/fr7.csv')
fr4m = pd.read_csv('sessmean/fr4.csv')

In [ ]:
fr1 = pd.merge(fr1,fr1m[['vid_id','framediff','framestd','framestdmean']],on='vid_id',how='left')
fr2 = pd.merge(fr2,fr2m[['vid_id','framediff','framestd','framestdmean']],on='vid_id',how='left')
fr3 = pd.merge(fr3,fr3m[['vid_id','framediff','framestd','framestdmean']],on='vid_id',how='left')
fr4 = pd.merge(fr4,fr4m[['vid_id','framediff','framestd','framestdmean']],on='vid_id',how='left')
fr5 = pd.merge(fr5,fr5m[['vid_id','framediff','framestd','framestdmean']],on='vid_id',how='left')
fr6 = pd.merge(fr6,fr6m[['vid_id','framediff','framestd','framestdmean']],on='vid_id',how='left')
fr7 = pd.merge(fr7,fr7m[['vid_id','framediff','framestd','framestdmean']],on='vid_id',how='left')
fr8 = pd.merge(fr8,fr8m[['vid_id','framediff','framestd','framestdmean']],on='vid_id',how='left')
fr9 = pd.merge(fr9,fr9m[['vid_id','framediff','framestd','framestdmean']],on='vid_id',how='left')

In [ ]:
fr = list(fr1.columns)

In [ ]:
fr8s = pd.read_csv('single/fr8.csv').add_suffix('single')
fr2s = pd.read_csv('single/fr2.csv').add_suffix('single')
fr1s = pd.read_csv('single/fr1.csv').add_suffix('single')
fr6s = pd.read_csv('single/fr6.csv').add_suffix('single')
fr9s = pd.read_csv('single/fr9.csv').add_suffix('single')
fr3s = pd.read_csv('single/fr3.csv').add_suffix('single')
fr5s = pd.read_csv('single/fr5.csv').add_suffix('single')
fr7s = pd.read_csv('single/fr7.csv').add_suffix('single')
fr4s = pd.read_csv('single/fr4.csv').add_suffix('single')

In [ ]:
fr1 = pd.merge(fr1,fr1s,left_on='vid_id',right_on='vid_idsingle',how='left')
fr2 = pd.merge(fr2,fr2s,left_on='vid_id',right_on='vid_idsingle',how='left')
fr3 = pd.merge(fr3,fr3s,left_on='vid_id',right_on='vid_idsingle',how='left')
fr4 = pd.merge(fr4,fr4s,left_on='vid_id',right_on='vid_idsingle',how='left')
fr5 = pd.merge(fr5,fr5s,left_on='vid_id',right_on='vid_idsingle',how='left')
fr6 = pd.merge(fr6,fr6s,left_on='vid_id',right_on='vid_idsingle',how='left')
fr7 = pd.merge(fr7,fr7s,left_on='vid_id',right_on='vid_idsingle',how='left')
fr8 = pd.merge(fr8,fr8s,left_on='vid_id',right_on='vid_idsingle',how='left')
fr9 = pd.merge(fr9,fr9s,left_on='vid_id',right_on='vid_idsingle',how='left')

In [ ]:
fr8g = pd.read_csv('group/fr8.csv').add_suffix('group')
fr2g = pd.read_csv('group/fr2.csv').add_suffix('group')
fr1g = pd.read_csv('group/fr1.csv').add_suffix('group')
fr6g = pd.read_csv('group/fr6.csv').add_suffix('group')
fr9g = pd.read_csv('group/fr9.csv').add_suffix('group')
fr3g = pd.read_csv('group/fr3.csv').add_suffix('group')
fr5g = pd.read_csv('group/fr5.csv').add_suffix('group')
fr7g = pd.read_csv('group/fr7.csv').add_suffix('group')
fr4g = pd.read_csv('group/fr4.csv').add_suffix('group')

In [ ]:
fr1 = pd.merge(fr1,fr1g,left_on='vid_id',right_on='vid_idgroup',how='left')
fr2 = pd.merge(fr2,fr2g,left_on='vid_id',right_on='vid_idgroup',how='left')
fr3 = pd.merge(fr3,fr3g,left_on='vid_id',right_on='vid_idgroup',how='left')
fr4 = pd.merge(fr4,fr4g,left_on='vid_id',right_on='vid_idgroup',how='left')
fr5 = pd.merge(fr5,fr5g,left_on='vid_id',right_on='vid_idgroup',how='left')
fr6 = pd.merge(fr6,fr6g,left_on='vid_id',right_on='vid_idgroup',how='left')
fr7 = pd.merge(fr7,fr7g,left_on='vid_id',right_on='vid_idgroup',how='left')
fr8 = pd.merge(fr8,fr8g,left_on='vid_id',right_on='vid_idgroup',how='left')
fr9 = pd.merge(fr9,fr9g,left_on='vid_id',right_on='vid_idgroup',how='left')

In [ ]:
df['train'] = 1
tdf['train'] = 0
tdf['target'] = np.nan
tdf = tdf[df.columns]
df = df.append(tdf)
del [[tdf]]
from ast import literal_eval
df.session_ids_and_duration = df.session_ids_and_duration.apply(literal_eval)
df = df.explode('session_ids_and_duration')
df.session_ids_and_duration = df.session_ids_and_duration.astype(str)
df.session_ids_and_duration = df.session_ids_and_duration.str[2:-1]
df[['session_id','duration']] = df.session_ids_and_duration.str.split('\': ',2,expand=True)
df.duration = df.duration.astype('int')
df=df.drop('session_ids_and_duration',axis=1)

In [ ]:
df['sess_count'] = df.groupby('session_id')['session_id'].transform('count')
df['max_duration'] = df.groupby('vid_id')['duration'].transform('max')
df['progress'] = df['duration']/df.max_duration
df['session_mean_progress'] = df.groupby('session_id')['progress'].transform('mean')
df = (df.groupby('vid_id').mean()).reset_index()
df

In [ ]:
frames = fr1.append(fr2).append(fr3).append(fr4).append(fr5).append(fr6).append(fr7).append(fr8).append(fr9)
frames['playratio'] = frames.vid_count / frames.session_mean
frames

In [ ]:
rp['v_count'] = rp.groupby(['vid_id'])['vid_id'].transform('count')
rp = rp.rename({'session_duration':'s_duration'},axis=1)
rp['s_count'] = rp.groupby(['session_id'])['session_id'].transform('count')
rp['vs_count'] = rp.groupby(['vid_id','session_id'])['vid_id'].transform('count')
rp = (rp.groupby('vid_id').mean()).reset_index()
rp

In [ ]:
frames['vid_count'] = frames['vid_count']/1000
frames['session_count'] = frames['session_count']/1000

In [ ]:
df = pd.merge(frames,df,on='vid_id',how='left')
df = pd.merge(df,clust,on='vid_id',how='left')
df = pd.merge(df,rp,on='vid_id',how='left')
drop_cols = ['vid_id','target','session_countsingle', 'vid_id','vid_idsingle','vid_idgroup','s_countsingle', 'v_fracsingle', 'count_ratiosingle', 'train']
cols = list(df.drop(drop_cols,axis=1).columns)
df

In [ ]:
df['a'] = df.frame/df.vsi_max
df['b'] = df.vidTime/df.vsi_max
df['c'] = df.session_count/df.vid_count
df['d'] = df.vidTime/df.session_duration
df['e'] = df.frame/df.session_duration
df['f'] = df.vsi_max/df.session_duration
df['g'] = df.timelineRatio-df.session_count
df['h'] = df.session_count/df.session_duration
df['i'] = df.count_ratio*df.session_count

In [ ]:
df['asingle'] = df.framesingle/df.vsi_maxsingle
df['bsingle'] = df.vidTimesingle/df.vsi_maxsingle
df['csingle'] = df.session_countsingle/df.vid_countsingle
df['dsingle'] = df.vidTimesingle/df.session_durationsingle
df['esingle'] = df.framesingle/df.session_durationsingle
df['fsingle'] = df.vsi_maxsingle/df.session_durationsingle
df['gsingle'] = df.timelineRatiosingle-df.session_countsingle
df['hsingle'] = df.session_countsingle/df.session_durationsingle
df['isingle'] = df.count_ratiosingle*df.session_countsingle

In [ ]:
df['agroup'] = df.framegroup/df.vsi_maxgroup
df['bgroup'] = df.vidTimegroup/df.vsi_maxgroup
df['cgroup'] = df.session_countgroup/df.vid_countgroup
df['dgroup'] = df.vidTimegroup/df.session_durationgroup
df['egroup'] = df.framegroup/df.session_durationgroup
df['fgroup'] = df.vsi_maxgroup/df.session_durationgroup
df['ggroup'] = df.timelineRatiogroup-df.session_countgroup
df['hgroup'] = df.session_countgroup/df.session_durationgroup
df['igroup'] = df.count_ratiogroup*df.session_countgroup

In [ ]:
df=df.replace({np.inf:np.nan})

In [ ]:
tdf = df[df.target.isna()]
df = df[df.target.notna()]
drop_cols = ['vid_id','target','session_countsingle', 'vid_id','vid_idsingle','vid_idgroup','s_countsingle', 'v_fracsingle', 'count_ratiosingle', 'train']
cols = list(df.drop(drop_cols,axis=1).columns)
le = LabelEncoder()
cv = KFold(n_splits=5)
cat_cols = list(df.columns[df.dtypes=='category'])
len(cols)

In [ ]:
model = XGBRegressor(n_estimators=25,objective='reg:tweedie')
tdf.target = model.fit(df[cols],df.target).predict(tdf[cols])
scores = cross_val_score(model,df[cols],df.target, scoring='neg_root_mean_squared_error',cv=cv, n_jobs=-1)
print(np.mean(scores))

In [ ]:
sub = pd.merge(ss['vid_id'],tdf[ss.columns],on='vid_id',how='left')
sub.to_csv('rhythm.csv',index=False)